# APPLICATION LEVEL TRAINING

### CONFIG VARS

In [1]:
from utils import *
from ultralytics import YOLO

In [2]:
DATASET_PATH="../Datasets/Dataset_ApplicationLevel"
PREPROCESSED_PATH="../Preprocessed_Datasets/ApplicationLevel"
AUGMENTED_PATH="../Augmented_Datasets/ApplicationLevel"
AUGMENTED_PATH_TRAIN_EX="../Augmented_Datasets/ApplicationLevel_train" # Data exclusively for training, not validating
YOLO_PATH="../YOLO_Datasets/ApplicationLevel"
YOLO_PATH_TRAIN_EX="../YOLO_Datasets/ApplicationLevel_train" # Data exclusively for training, not validating

In [3]:
if not os.path.exists(AUGMENTED_PATH):
    os.makedirs(AUGMENTED_PATH)
if not os.path.exists(AUGMENTED_PATH_TRAIN_EX):
    os.makedirs(AUGMENTED_PATH_TRAIN_EX)

### AUGMENTATION

For this model we will apply the following augmentation techniques:
- Hue transformations (-100º to +100º)
- Contrast inversion (To simulate dark and light modes)

In [4]:
resize_dataset_images(DATASET_PATH, AUGMENTED_PATH, 640, 360)

In [5]:
hue_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15, 100)

In [6]:
contrast_inversion_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15)

In [7]:
# Add the augmented data to the train exclusively folder
for file in os.listdir(AUGMENTED_PATH):
    shutil.copy(os.path.join(AUGMENTED_PATH, file), AUGMENTED_PATH_TRAIN_EX)

### FORMAT CONVERSION

Up to now, we have treated with labelme format datasets, but we need to convert it to YOLOv8 format if we want to train
a model, which has the following format:

```
YOLOv8_Dataset/
├── data.yaml
├── train/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
├── valid/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
└── test/ (OPTIONAL)
    ├── images/
    │   ├── img1.jpg
    │   ├── img2.jpg
    │   └── ...
    └── labels/
        ├── img1.txt
        ├── img2.txt
        └── ...
```

The format of the data.yml file is:
```
path: <path_to_dataset_root_dit>
train: <path_to_train_images>
val: <path_to_validation_images>
test: <path_to_test_images> (OPTIONAL)

nc: <number_of_classes>
names: ['class1', 'class2', 'class3', ...]
```

The labels for Instance segmentation have the following format for each annotation:
```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```

In [8]:
labelme_to_yolo(AUGMENTED_PATH_TRAIN_EX, YOLO_PATH_TRAIN_EX, 0.7,[
                    "Header",
                    "Toolbar",
                    "BrowserToolbar",
                    "Scrollbar",
                ], "seg")

### TRAIN

We will perform fine-tuning over the mobile-sam model using the hyperparameter tuning provided by Ultralytics to get the
best results we can. Since this is a non-standard dataset in terms of object features it is not clear what are the values
we should use.

We will also configure the training to not do any augmentation over the train set

In [1]:
from ultralytics import YOLO
# Initialize the YOLO model
model = YOLO("yolov8s-seg")

In [10]:
import torch
# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {} device'.format(device))

Using cuda device


In [3]:
# Tune hyperparameters on dataset for 30 epochs
model.tune(data=f"{YOLO_PATH_TRAIN_EX}/data.yaml", workers=1, iterations=20, epochs=30, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0)

Tuner: Initialized Tuner instance with 'tune_dir=runs\segment\tune14'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/20 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


Saved runs\segment\tune14\tune_scatter_plots.png
Saved runs\segment\tune14\tune_fitness.png

Tuner: 1/20 iterations complete  (2864.86s)
Tuner: Results saved to runs\segment\tune14
Tuner: Best fitness=0.09559 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.4018, 'metrics/recall(B)': 0.22542, 'metrics/mAP50(B)': 0.14118, 'metrics/mAP50-95(B)': 0.04771, 'metrics/precision(M)': 0.4473, 'metrics/recall(M)': 0.13198, 'metrics/mAP50(M)': 0.1043, 'metrics/mAP50-95(M)': 0.03122, 'val/box_loss': 2.72674, 'val/seg_loss': 2.14629, 'val/cls_loss': 2.55123, 'val/dfl_loss': 1.12117, 'fitness': 0.09559}
Tuner: Best fitness model is runs\segment\train99
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs\segment\tune14\best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
box: 7.5
cls: 0.5
dfl: 1.5
hsv_h: 0.0
hsv_s: 0.0
hsv_v: 0.0
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.

In [4]:
model = YOLO("runs/segment/tune14/weights/best.pt")

In [5]:
# Validate the model
metrics = model.val(workers=1)  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.208  Python-3.9.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
YOLOv8s-seg summary (fused): 195 layers, 11781148 parameters, 0 gradients, 42.4 GFLOPs
val: Scanning D:\Code\Screen2SOM-Training\YOLO_Datasets\ApplicationLevel_train\val\labels.cache... 41 images, 1 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:45<00:00, 15.32s/it]
                   all         41        143      0.842      0.251        0.3      0.126      0.853      0.189       0.22      0.079
                Header         41         51      0.711      0.338      0.488      0.232      0.739      0.255      0.343      0.109
               Toolbar         41         15          1          0     0.0241    0.00938          1          0     0.0191    0.00873
        BrowserToolbar         41         24      0.65

In [11]:
metrics.seg.map    # map50-95
metrics.seg.map50  # map50
metrics.seg.map75  # map75
metrics.seg.maps   # a list contains map50-95 of each category

array([    0.19274,     0.32869,     0.40419,     0.39927,       0.561,     0.28032,     0.10029,     0.16814,     0.26469,     0.19651,     0.35886,     0.31838,     0.15703])

In [12]:
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

array([    0.44217,     0.57988,      0.5628,     0.66923,     0.72803,     0.39053,     0.33478,     0.53026,     0.41717,     0.49925,     0.55777,     0.43547,      0.2592])

In [10]:
inference = model("../YOLO_Datasets/Elementlevel_train/val/images/Captura de pantalla (44)_2.jpg", save=True)


image 1/1 d:\Code\Screen2SOM-Training\Training\..\YOLO_Datasets\Elementlevel_train\val\images\Captura de pantalla (44)_2.jpg: 384x640 33 WebIcons, 4 Icons, 3 BtnPills, 3 TextInputs, 2 Links, 29.0ms
Speed: 3.0ms preprocess, 29.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\segment\predict


In [1]:
# Load Mobile-SAM
from ultralytics import SAM

model = SAM("mobile_sam.pt")

In [4]:
# Tune hyperparameters on dataset for 30 epochs
model.tune(data="../YOLO_Datasets/ElementLevel_train/data.yaml", workers=1, epochs=30, iterations=20, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, translate=0.0, scale=0.0, fliplr=0.0, mosaic=0.0)

Tuner: Initialized Tuner instance with 'tune_dir=runs\segment\tune13'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/20 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.0, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.0, 'mosaic': 0.0, 'mixup': 0.0, 'copy_paste': 0.0}
